#  IMPORTING LIBRARIES 


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests

# LOADING WEB PAGES WITH 'REQUEST'

In [3]:
url = "https://www.w3schools.com/html/default.asp"

# Lets make a request to "https://www.w3schools.com/html/default.asp"
# Will store the result in 'res' variable
r = requests.get(url)

txt = r.text
status = r.status_code

In [4]:
#printing the staus of the requests
print(status)

200


So it's a successful request

# EXTRACTING TITLE WITH BEAUTIFULSOUP

In [5]:
page = requests.get("https://www.w3schools.com/html/default.asp")
soup = BeautifulSoup(page.content, 'html.parser')

In [6]:
# Extracting title of page
page_title = soup.title.text

# prining the result
print(page_title)

HTML Tutorial


# SOUP-ED BODY AND HEAD 

In [7]:
# Extracting body of page
page_body = soup.body

# Extracting head of page
page_head = soup.head

# printing the result
print(page_head)

<head>
<title>HTML Tutorial</title>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="HTML, Python, CSS, SQL, JavaScript, How to, PHP, Java, C++, jQuery, Bootstrap, C#, Colors, W3.CSS, XML, MySQL, Icons, NodeJS, React, Graphics, Angular, R, AI, Git, Data Science, Code Game, Tutorials, Programming, Web Development, Training, Learning, Quiz, Courses, Lessons, References, Examples, Source code, Demos, Tips" name="Keywords"/>
<meta content="Well organized and easy to understand Web building tutorials with lots of examples of how to use HTML, CSS, JavaScript, SQL, Python, PHP, Bootstrap, Java, XML and more." name="Description"/>
<meta content="https://www.w3schools.com/images/w3schools_logo_436_2.png" property="og:image"/>
<meta content="image/png" property="og:image:type"/>
<meta content="436" property="og:image:width"/>
<meta content="228" property="og:image:height"/>
<meta content="W3Schools offers free online tutorials, referenc

# SELECT WITH BEAUTIFULSOUP

In [8]:
# Extracting first <h1>(...)</h1> text
first_h1 = soup.select('h1')[0].text 
print(first_h1)

HTML Tutorial


# PASSING REQUIRMENTS 

In [8]:
# Creating all_h1_tags as empty list
all_h1_tags = []

# Set all_h1_tags to all h1 tags of the soup
for element in soup.select('h1'):
    all_h1_tags.append(element.text)

# Create seventh_p_text and set it to 7th p element text of the page
seventh_p_text = soup.select('p')[7].text

print(all_h1_tags, seventh_p_text)

['HTML Tutorial'] This HTML tutorial also contains nearly 100 HTML exercises.


# EXTRACTING LINKS

In [9]:
# Create top_items as empty list
all_links = []

# Extract and store in top_items according to instructions on the left
links = soup.select('a')
for ahref in links:
    text = ahref.text
    text = text.strip() if text is not None else ''

    href = ahref.get('href')
    href = href.strip() if href is not None else ''
    all_links.append({"href": href, "text": text})

print(all_links)

[{'href': 'https://www.w3schools.com', 'text': ''}, {'href': 'javascript:void(0)', 'text': 'Tutorials'}, {'href': 'javascript:void(0)', 'text': 'References'}, {'href': 'javascript:void(0)', 'text': 'Exercises'}, {'href': 'https://www.w3schools.com/videos/index.php', 'text': 'Videos NEW'}, {'href': 'javascript:void(0)', 'text': 'Menu'}, {'href': 'https://profile.w3schools.com/log-in?redirect_url=https%3A%2F%2Fmy-learning.w3schools.com', 'text': 'Log in'}, {'href': 'https://courses.w3schools.com/', 'text': 'Paid Courses'}, {'href': 'https://www.w3schools.com/spaces', 'text': 'Website NEW'}, {'href': 'javascript:void(0);', 'text': ''}, {'href': '/default.asp', 'text': ''}, {'href': '/html/default.asp', 'text': 'HTML'}, {'href': '/css/default.asp', 'text': 'CSS'}, {'href': '/js/default.asp', 'text': 'JAVASCRIPT'}, {'href': '/sql/default.asp', 'text': 'SQL'}, {'href': '/python/default.asp', 'text': 'PYTHON'}, {'href': '/php/default.asp', 'text': 'PHP'}, {'href': '/bootstrap/bootstrap_ver.as

In [10]:
# extracting all the URLs found within a page’s <a> tags
for link in soup.find_all('a'):
    print(link.get('href'))

https://www.w3schools.com
javascript:void(0)
javascript:void(0)
javascript:void(0)
https://www.w3schools.com/videos/index.php
javascript:void(0)
https://profile.w3schools.com/log-in?redirect_url=https%3A%2F%2Fmy-learning.w3schools.com
https://courses.w3schools.com/
https://www.w3schools.com/spaces
javascript:void(0);
/default.asp
/html/default.asp
/css/default.asp
/js/default.asp
/sql/default.asp
/python/default.asp
/php/default.asp
/bootstrap/bootstrap_ver.asp
/howto/default.asp
/w3css/default.asp
/java/default.asp
/jquery/default.asp
/cpp/default.asp
/cs/index.php
/r/default.asp
/react/default.asp
/kotlin/index.php
javascript:void(0);
javascript:void(0);
javascript:void(0);
/html/default.asp
/css/default.asp
/css/css_rwd_intro.asp
/bootstrap/bootstrap_ver.asp
/w3css/default.asp
/colors/default.asp
/icons/default.asp
/graphics/default.asp
/graphics/svg_intro.asp
/graphics/canvas_intro.asp
/howto/default.asp
/sass/default.php
/ai/default.asp
/python/python_ml_getting_started.asp
/datas

# MAIN HEADINGS OF HOME PAGE 

In [12]:
# creating a list of all common heading tags
heading_tags = ["h1","h2","h3"]
for tags in soup.find_all(heading_tags):
    print(tags.name + ' -> ' + tags.text.strip())

h2 -> Tutorials
h3 -> HTML and CSS
h3 -> Data Analytics
h3 -> XML Tutorials
h3 -> JavaScript
h3 -> Programming
h3 -> Server Side
h3 -> Web Building
h3 -> Data Analytics
h3 -> XML Tutorials
h2 -> References
h3 -> HTML
h3 -> CSS
h3 -> JavaScript
h3 -> Programming
h3 -> Server Side
h3 -> XML
h3 -> Character Sets
h2 -> Exercises and Quizzes
h3 -> Exercises
h3 -> Quizzes
h3 -> Courses
h3 -> Certificates
h2 -> HTML Tutorial
h2 -> HTML Forms
h2 -> HTML Graphics
h2 -> HTML Media
h2 -> HTML APIs
h2 -> HTML Examples
h2 -> HTML References
h1 -> HTML Tutorial
h2 -> Easy Learning with HTML "Try it Yourself"
h3 -> Example
h2 -> HTML Examples
h2 -> HTML Exercises
h2 -> Test Yourself With Exercises
h2 -> Exercise:
h2 -> HTML Quiz Test
h2 -> HTML References
h2 -> Kickstart your career
h2 -> Report Error
h2 -> Thank You For Helping Us!


# SUB HEADINGS OF HTML 

In [11]:
heading1 = soup.find_all('h2', class_ = 'left')
heading1

[<h2 class="left"><span class="left_h2">HTML</span> Tutorial</h2>,
 <h2 class="left"><span class="left_h2">HTML</span> Forms</h2>,
 <h2 class="left"><span class="left_h2">HTML</span> Graphics</h2>,
 <h2 class="left"><span class="left_h2">HTML</span> Media</h2>,
 <h2 class="left"><span class="left_h2">HTML</span> APIs</h2>,
 <h2 class="left"><span class="left_h2">HTML</span> Examples</h2>,
 <h2 class="left"><span class="left_h2">HTML</span> References</h2>]

In [12]:
sub_headings = []
for i in heading1:
    print(i.text)
    sub_headings.append(i.text)

HTML Tutorial
HTML Forms
HTML Graphics
HTML Media
HTML APIs
HTML Examples
HTML References


In [13]:
# contents in html
heading2 = soup.find_all('a', target='_top')
heading2

[<a class="w3-bar-item w3-button" href="/html/html_quiz.asp" target="_top">HTML Quiz</a>,
 <a class="w3-bar-item w3-button" href="/css/css_quiz.asp" target="_top">CSS Quiz</a>,
 <a class="w3-bar-item w3-button" href="/js/js_quiz.asp" target="_top">JavaScript Quiz</a>,
 <a class="w3-bar-item w3-button" href="/sql/sql_quiz.asp" target="_top">SQL Quiz</a>,
 <a class="w3-bar-item w3-button" href="/mysql/mysql_quiz.asp" target="_top">MySQL Quiz</a>,
 <a class="w3-bar-item w3-button" href="/php/php_quiz.asp" target="_top">PHP Quiz</a>,
 <a class="w3-bar-item w3-button" href="/python/python_quiz.asp" target="_top">Python Quiz</a>,
 <a class="w3-bar-item w3-button" href="/python/numpy/numpy_quiz.asp" target="_top">NumPy Quiz</a>,
 <a class="w3-bar-item w3-button" href="/python/pandas/pandas_quiz.asp" target="_top">Pandas Quiz</a>,
 <a class="w3-bar-item w3-button" href="/python/scipy/scipy_quiz.php" target="_top">SciPy Quiz</a>,
 <a class="w3-bar-item w3-button" href="/jquery/jquery_quiz.asp" 

# Lets scrape the data for the column 'TOPICS', 'LINKS', and 'DESCRIPTION'

----

In [15]:
course_contents = []
course_links = []
base_url = "https://www.w3schools.com/html/"
for i in heading2:
    if 'HTML' in i.text and i.text!='HTML Quiz':
        course_contents.append(i.text)
        extra = i.get('href')
        if extra[0] == '/':
            base_url = "https://www.w3schools.com"
        course_links.append((base_url + extra))

In [16]:
# getting the range length
length = len(course_contents)

In [17]:
print("Course content ---->>>> Course link\n----------------------------------------------------------------")
for i in range(length):
    print(f'{course_contents[i]} ----->>>> {course_links[i]}')

Course content ---->>>> Course link
----------------------------------------------------------------
HTML HOME ----->>>> https://www.w3schools.com/html/default.asp
HTML Introduction ----->>>> https://www.w3schools.com/html/html_intro.asp
HTML Editors ----->>>> https://www.w3schools.com/html/html_editors.asp
HTML Basic ----->>>> https://www.w3schools.com/html/html_basic.asp
HTML Elements ----->>>> https://www.w3schools.com/html/html_elements.asp
HTML Attributes ----->>>> https://www.w3schools.com/html/html_attributes.asp
HTML Headings ----->>>> https://www.w3schools.com/html/html_headings.asp
HTML Paragraphs ----->>>> https://www.w3schools.com/html/html_paragraphs.asp
HTML Styles ----->>>> https://www.w3schools.com/html/html_styles.asp
HTML Formatting ----->>>> https://www.w3schools.com/html/html_formatting.asp
HTML Quotations ----->>>> https://www.w3schools.com/html/html_quotation_elements.asp
HTML Comments ----->>>> https://www.w3schools.com/html/html_comments.asp
HTML Colors ----->>>

# CONTENT PART OF PER PAGE OF THE WEBSITE 

In [39]:
sub_page_content_list = []

for i in range(length):
    page_url = course_links[i]
    sub_page = requests.get(page_url)
    soup_page = BeautifulSoup(sub_page.content, 'html.parser')
    
    page_content = soup_page.find_all('div', class_ = 'w3-col l10 m12', id='main')
#     print(page_content)
    for page_data in page_content: 
#         print(page_data)
        page_paragraphs = page_data.find_all('p')
#         print(page_paragraphs)
    content_per_page = []
    for i in page_paragraphs:
        content_per_page.append(i.text)
    sub_page_content_list.append(content_per_page)
    time.sleep(5)

In [40]:
sub_page_content_list

[['HTML is the standard markup language for Web pages.',
  'With HTML you can create your own Website.',
  'HTML is easy to learn - You will enjoy it!',
  'With our "Try it Yourself" editor, you can edit the HTML code and view the \nresult:',
  'Click on the "Try it Yourself" button to see how it works.',
  'In this HTML tutorial, you will find more than 200 examples. With our online \n"Try it Yourself" editor, you can edit and test each example yourself!',
  'Go to HTML Examples!',
  'This HTML tutorial also contains nearly 100 HTML exercises.',
  'Add a "tooltip" to the paragraph below with the text "About W3Schools".',
  'Start the Exercise',
  'Test your HTML skills with our HTML Quiz!',
  'Start HTML Quiz!',
  'At W3Schools you will find complete references about HTML elements, \nattributes, events, color names, entities, character-sets, URL encoding, \nlanguage codes, HTTP messages, browser support, and more:',
  'Get certified by completing the  course'],
 ['HTML is the standard

In [43]:
from csv import writer

with open('html_w3school.csv', 'w', encoding='utf8', newline='') as f:
    the_writer = writer(f)
    header = ['Topics', 'Links', 'Description']
    the_writer.writerow(header)
    for i in range(length):
        row = [course_contents[i], course_links[i], sub_page_content_list[i]]
        the_writer.writerow(row)

In [44]:
file = pd.read_csv('html_w3school.csv')
dataset = pd.DataFrame(file)

In [45]:
dataset

,Topics,Links,Description
0,HTML HOME,https://www.w3schools.com/html/default.asp,['HTML is the standard markup language for Web...
1,HTML Introduction,https://www.w3schools.com/html/html_intro.asp,['HTML is the standard markup language for cre...
2,HTML Editors,https://www.w3schools.com/html/html_editors.asp,['A simple text editor is all you need to lear...
3,HTML Basic,https://www.w3schools.com/html/html_basic.asp,['In this chapter we will show some basic HTML...
4,HTML Elements,https://www.w3schools.com/html/html_elements.asp,"['An HTML element is defined by a start tag, s..."
...,...,...,...
68,HTML Audio/Video,https://www.w3schools.com/tags/ref_av_dom.asp,"['The HTML5 DOM has methods, properties, and e..."
69,HTML Doctypes,https://www.w3schools.com/tags/ref_html_dtd.asp,['All HTML documents must start with a \n<!DOC...
70,HTML Character Sets,https://www.w3schools.com/tags/ref_characterse...,['The default character set in HTML5 is UTF-8....
71,HTML URL Encode,https://www.w3schools.com/tags/ref_urlencode.asp,['Web browsers request pages from web servers ...
